## Course: TM10007 - Machine learning
Editors: Lishia Vergeer, Amy Roos, Maaike Pruijt, Hilde Roording.

Description: The aim of this code is to predict the tumor grade of glioma’s(high or low) before surgery, 
based on features extracted from a combination of four MRI images: 
T2-weighted, T2-weighted FLAIR and T1-weighted before and after injection of contrast agent.

#### Import packages

In [183]:
# General packages
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets as ds

# Import code
from brats.load_data import load_data

# Performance 
from sklearn.model_selection import train_test_split
from sklearn import decomposition
import seaborn


# Pipeline and gridsearch
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV

#preprocessing
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
# scaler
from sklearn.preprocessing import RobustScaler

#Machine learning classifiers
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn import feature_selection 
from sklearn import preprocessing
from sklearn import neighbors
from sklearn.model_selection import ShuffleSplit

import xgboost as xgb
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn import svm
from sklearn.model_selection import cross_val_score, KFold
from sklearn.neighbors import KNeighborsRegressor



#### Load data

In [184]:
data_brats = load_data()

# Convert to dataframe
X = pd.DataFrame(data_brats)

# Print data 
print(f'The number of samples in data_brats: {len(X.index)}')
print(f'The number of columns in data_brats: {len(X.columns)}')

The number of samples in data_brats: 167
The number of columns in data_brats: 725


#### Split data in X and y
Split in X (data) and y (label)

In [185]:
# split column label from dataset X
y = X.pop('label')

#### Split data in train and test set
This function creates a panda dataframe and splits the data into test and train components.
This is done with test_size variable and the function train_test_split from the sklearn module.
Returns a train set with the data of 80% and a test set of 20% of the subjects.



In [186]:
# Split data in train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2)  


#### Preprocessing : 

In [187]:
# infinity to NaN
X_train[X_train==np.inf]=np.nan
X_test[X_test==np.inf]=np.nan

# non-numeric features to NaN
X_train = X_train.replace(['#DIV/0!'], np.nan)
X_train = X_train.apply(pd.to_numeric, errors='coerce')

X_test  = X_test.replace(['#DIV/0!'], np.nan)
X_test = X_test.apply(pd.to_numeric, errors='coerce')

# If the total number of NaN observations in a column are greater than 40%, delete the entire column.
perc = 40.0
min_count = int(((100-perc)/100)*X_train.shape[0] + 1)
X_train_drop = X_train.dropna(axis=1, thresh=min_count)
X_labels = X_train_drop.keys()

X_test_drop = X_test[X_labels]

# fill the NaN observations.
data_fill_train = X_train_drop.fillna(X_train_drop.mean()) 
data_fill_test = X_test_drop.fillna(X_test_drop.mean()) 

#### Scale features

In [188]:
# robustscaler
scaler = RobustScaler()
scaler.fit(data_fill_train)
X_train_scaled = scaler.transform(data_fill_train)
X_test_scaled = scaler.transform(data_fill_test)

#### Transform features

In [189]:
# Perform a PCA
pca = decomposition.PCA(n_components=5)
pca.fit(X_train_scaled) 
X_train_pca = pca.transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)


#### Cross validation

In [190]:
# Create a cross-validation object

cv = KFold(n_splits=5, shuffle= True, random_state = 1)

X = X_train_pca

y = y_train.values
y = np.where(y=='GBM', 1, y)
y = np.where(y=='LGG', 2, y)
y = y.tolist()

classifiers = (KNeighborsClassifier(), RandomForestClassifier(), SVC(kernel="rbf", C=0.025, probability=True), NuSVC(probability=True), DecisionTreeClassifier(), AdaBoostClassifier(), GradientBoostingClassifier() )

for cls in classifiers:

    list_scores = cross_val_score(cls, X, y, scoring = 'accuracy', cv=cv)
    mean = np.mean(list_scores)

    print(list_scores)
    print(f'{cls} mean:', mean)
  


[0.59259259 0.7037037  0.7037037  0.65384615 0.88461538]
KNeighborsClassifier() mean: 0.7076923076923076
[0.59259259 0.85185185 0.62962963 0.65384615 0.76923077]
RandomForestClassifier() mean: 0.6994301994301994
[0.55555556 0.7037037  0.55555556 0.61538462 0.57692308]
SVC(C=0.025, probability=True) mean: 0.6014245014245014
[0.62962963 0.74074074 0.59259259 0.53846154 0.65384615]
NuSVC(probability=True) mean: 0.631054131054131
[0.51851852 0.66666667 0.66666667 0.65384615 0.84615385]
DecisionTreeClassifier() mean: 0.6703703703703703
[0.62962963 0.62962963 0.62962963 0.61538462 0.80769231]
AdaBoostClassifier() mean: 0.6623931623931624
[0.59259259 0.7037037  0.66666667 0.65384615 0.88461538]
GradientBoostingClassifier() mean: 0.7002849002849002


#### Classifier

In [191]:
# Construct classifiers
clsfs = [RandomForestClassifier(n_estimators=1, random_state=42),
         RandomForestClassifier(n_estimators=5, random_state=42),
         RandomForestClassifier(n_estimators=200, random_state=42)]


